Dynamic Graph Neural Ordinary Differential Equation Network for
Multi-modal Emotion Recognition in Conversation

摘要

Multimodal emotion recognition in conversation (MERC) :多模态情感识别对话

指出现在大多数多模态情感识别方法采用GCN， 存在问题1过拟合，2无法捕捉情感的时序依赖

本文提出的是DGODE，利用自适应混合跳跃（mixhop）机制提升GCN的泛化能力，并通过图ODE演化网络来刻画节点表示随时间的连续动态变化，从而捕捉时序依赖性。还能缓解过平滑问题，从而支持构建更深层次的GCN网络。实际上这个也就是贡献

In [ ]:
Intro

多模态情感识别对话（MERC）整合多种模态的数据显著提升了情感识别的准确性和广泛应用性，本文发现现有GCN模型通常只包含4层GCN，而随着层数的增加，性能会显著下降。原因可能在于，基础GCN中的信息聚合仅仅是邻域内的简单消息平滑，导致随着层数的堆叠，邻近节点趋于同一数值。

本文模型DGODE在GCN层数增加时，仍能表现出稳定的性能。

2

2.1没有什么内容

2.2CGNN开发了一种连续消息传递层，以实现节点状态的连续动态建模。与传统的图神经网络（GCN）不同，CGNN不再依赖于固定层数的信息传播，而是通过求解常微分方程，使节点之间的信息能够连续传播。CGNN还引入了重启分布，在信息传播过程中及时将节点状态“重置”为初始状态，从而避免了过平滑问题。



**3.1 图神经网络（Graph Neural Networks）**

给定一个图 $G = (V, E)$， $V$ 是节点的集合，$E$ 是边的集合。

每个节点 $v \in V$ 构成节点特征矩阵 $X \in \mathbb{R}^{|V| \times d}$，其中 $d$ 表示特征的维度。$X$ 的每一行对应一个节点的特征表示。

使用二值邻接矩阵 $A \in \mathbb{R}^{|V| \times |V|}$ 来表示节点 $i$ 与节点 $j$ 之间的连接关系。如果 $a_{ij} = 1$，则表示节点 $i$ 和节点 $j$ 之间有一条边；如果 $a_{ij} = 0$，则表示两者之间没有连接。

目标是学习一个节点表示矩阵 $H$，该矩阵能够捕捉到图中节点的结构信息和特征信息。

对称归一化邻接矩阵：

$$
\hat{A} = \alpha \left( I + D^{-\frac{1}{2}} A D^{-\frac{1}{2}} \right)
$$

其中 $\alpha$ 是一个超参数。

---

**3.2 神经常微分方程（Neural Ordinary Differential Equation, Neural ODE）**

一个输入数据 $x(t)$，其演化过程可用如下ODE形式描述：

$$
\frac{dx(t)}{dt} = f(x(t), t, \theta)
$$

其中，$x(t)$ 表示时刻 $t$ 的隐藏状态，$f$ 是带参数 $\theta$ 的神经网络函数。

---

**3.3 多模态特征提取**

* **词嵌入（Word Embedding）：** 按照以往研究（Chudasama 等，2022；Li 等，2022b），本文采用 RoBERTa（Liu，2019）来获得文本的上下文嵌入表示。
* **视觉与音频特征提取（Visual and Audio Feature Extraction）：** 参考以往工作（Ma 等，2023；Lian 等，2021），我们选用了 DenseNet（Huang 等，2017）和 openSMILE（Eyben 等，2010）作为视频和音频的特征提取工具。




输入：一段包含多轮对话的对话序列，每轮有说话人和三种模态（音频、视频、文本）特征。

输出：每一句话对应的情感标签。

任务难点：需要同时利用三种数据模态的信息，还要考虑对话中多说话人、时序、上下文等复杂因素。

4.1 Modality Encoding

建模时不仅要有每句话的内容、音频、视频等信息，还把说话人身份和上下文语境信息编码进去。

每个说话人用一个独特的one-hot向量 $p_i$ 表示。用一个可学习的权重矩阵 $W_p$ 对 $p_i$ 做线性变换，得到说话人嵌入向量 $P_i = W_p p_i$。

对于文本、音频、视频三种模态，分别用GRU对序列数据进行建模，抓取其时序上下文特征。

即对每一轮发言的每种模态输入，GRU都会考虑“这句话前后的内容”，提取时间相关性和上下文依赖信息。

公式写作：$\hat{v_m^i} = \overleftrightarrow{GRU}(v_m^i, c_m^i(+,-)), m \in \{a, v, f\}$

  其中 $v_m^i$ 是第i轮的模态输入，$c_m^i(+,-)$ 是GRU的cell状态（带正反向）。
  
  m = a（audio），v（visual），f（feature or fused, 实际可理解为文本）。

得到每种模态的上下文特征（GRU输出），再加上当前说话人的嵌入向量，形成融合了说话人身份和上下文的单模态表示：

  * $h_m^i = c_m^i + S_i, m \in \{t, a, v\}$
  * 其中 $S_i$ 就是上面算出来的说话人嵌入，$c_m^i$ 是模态特征。



**传统GCN的局限**：
  传统GCN**每一层只能聚合“一阶邻居”**（即相连的直接邻居），即使堆叠多层也往往聚合能力有限，且容易出现“过平滑”（所有节点特征越来越相似）。


*MixHop思想：多阶邻居聚合


MixHop就是**让每一层可以同时聚合多阶邻居**的信息，比如一阶、二阶、三阶……甚至更远的邻居节点。通俗地说，MixHop可以“多跳”收集信息，而不是只看最邻近的节点。

文中强调“自适应”，即模型可以自学习/自选择每阶邻居聚合时的权重或方式，更灵活地调整信息融合策略。

**公式**

$$
H_{n+1} = \sum_{n=1}^{N} \hat{A}^{n} H_n W + H_0
$$

* $\hat{A}$：归一化的邻接矩阵
* $\hat{A}^n$：表示n阶邻居的信息扩散（比如$\hat{A}^2$就是二阶邻居）
* $H_n$：第n层的节点特征
* $W$：可学习的权重参数（对每阶聚合线性变换）
* $H_0$：原始节点特征（残差项，类似ResNet的设计，防止信息丢失）


4.3 Temporal Graph ODE

MixHop GCN虽然能够捕捉多阶邻居关系，但它本质还是**离散的层级传播**，**无法模拟节点表示随时间连续变化的动态过程**。而在多模态情感识别中，情绪是随时间动态演变的，单纯的离散GCN难以捕捉“情感随时间流动的连续依赖”。


 **把GCN的信息传播过程，从“离散层级”推广为“连续动态演化”**，即用常微分方程（ODE）来建模节点状态随时间的连续变化。这样，模型就可以捕捉情绪/节点特征随时间推移的细腻变化过程。

1. **离散到连续的转化**

   * 之前的GCN聚合公式是离散的：

     $$
     H_{n+1} = \sum_{n=1}^{N} \hat{A}^{n} H_n W + H_0
     $$
   * 现在把它看作是“积分的Riemann和”。本质上，把离散相加看作积分的近似，从而推广到连续时间域。

2. **引入ODE建模传播过程**

   * 推导得出，节点特征 $H(t)$ 的连续动态演化可以表示为一个ODE（常微分方程）：

     $$
     \frac{dH(t)}{dt} = \frac{1}{N}\sum_{n=1}^{N} \Big( \ln\hat{A} H(t) + H(t)\ln W + E \Big)
     $$

     * $H(t)$：时刻t的节点特征
     * $\ln\hat{A}$：对归一化邻接矩阵做对数，理论上是为了便于连续化传播建模
     * $W$：权重矩阵
     * $E$：初始状态相关项

   * 这样建模后，信息传播就不是单步单层，而是**时间上连续地演化，能够模拟“情感随时间流转”的细节**。

3. **解析解与数值求解**

   * ODE在某些情况下的解析解（公式8），不过实际用法里更常见的是用\*\*ODE求解器（ODESolver）\*\*在数值上逐步模拟节点特征的变化过程：

     $$
     H(t) = ODESolver\left(\frac{dH(t)}{dt}, H_0, t\right)
     $$

     * $H_0$：初始节点特征
     * ODESolver可以选用如Euler、RK4等常用数值解法